In [0]:
#criando um dataframe com dados fixos
dados =[("Grimaldo","Oliveira","Brasileira","Professor","M",3000),
        ("Ana ","Santos","Portuguesa","Atriz","F",4000),
        ("Roberto","Carlos","Brasileira","Analista","M",4000),
        ("Maria","Santanna","Italiana","Dentista","F",6000),
        ("Jeane","Andrade","Inglesa","Medica","F",7000)]

colunas=["Primeiro_Nome","Ultimo_nome","Nacionalidade","Trabalho","Genero","Salario"]
datafparquet=spark.createDataFrame(dados,colunas)
datafparquet.show()

+-------------+-----------+-------------+---------+------+-------+
|Primeiro_Nome|Ultimo_nome|Nacionalidade| Trabalho|Genero|Salario|
+-------------+-----------+-------------+---------+------+-------+
|     Grimaldo|   Oliveira|   Brasileira|Professor|     M|   3000|
|         Ana |     Santos|   Portuguesa|    Atriz|     F|   4000|
|      Roberto|     Carlos|   Brasileira| Analista|     M|   4000|
|        Maria|   Santanna|     Italiana| Dentista|     F|   6000|
|        Jeane|    Andrade|      Inglesa|   Medica|     F|   7000|
+-------------+-----------+-------------+---------+------+-------+



In [0]:
#criando o arquivo parquet
datafparquet.write.parquet("/FileStore/tables/parquet/pessoal.parquet")

In [0]:

#Permite uma atualização do arquivo parquet
datafparquet.write.mode('overwrite').parquet('/FileStore/tables/parquet/pessoal.parquet')


In [0]:
#Realizando uma atualização do arquivo parquet
datafleitura=spark.read.parquet("/FileStore/tables/parquet/pessoal.parquet")
datafleitura.show()


+-------------+-----------+-------------+---------+------+-------+
|Primeiro_Nome|Ultimo_nome|Nacionalidade| Trabalho|Genero|Salario|
+-------------+-----------+-------------+---------+------+-------+
|     Grimaldo|   Oliveira|   Brasileira|Professor|     M|   3000|
|      Roberto|     Carlos|   Brasileira| Analista|     M|   4000|
|        Maria|   Santanna|     Italiana| Dentista|     F|   6000|
|         Ana |     Santos|   Portuguesa|    Atriz|     F|   4000|
|        Jeane|    Andrade|      Inglesa|   Medica|     F|   7000|
+-------------+-----------+-------------+---------+------+-------+



In [0]:
#Criando uma consulta em SQL
datafleitura.createOrReplaceTempView("Tabela_Parquet")
ResultSQL = spark.sql("select * from Tabela_Parquet where salario >= 6000 ")
ResultSQL.show()

+-------------+-----------+-------------+--------+------+-------+
|Primeiro_Nome|Ultimo_nome|Nacionalidade|Trabalho|Genero|Salario|
+-------------+-----------+-------------+--------+------+-------+
|        Maria|   Santanna|     Italiana|Dentista|     F|   6000|
|        Jeane|    Andrade|   Portuguesa|  Medica|     F|   7000|
+-------------+-----------+-------------+--------+------+-------+



## Utilizando Partições


In [0]:
#Particionando os dados em um arquivo parquet
datafparquet.write.partitionBy("Nacionalidade","salario").mode("overwrite").parquet("/FileStore/tables/parquet/pessoal.parquet")
display(datafparquet)

Primeiro_Nome,Ultimo_nome,Nacionalidade,Trabalho,Genero,Salario
Grimaldo,Oliveira,Brasileira,Professor,M,3000
Ana,Santos,Portuguesa,Atriz,F,4000
Roberto,Carlos,Brasileira,Analista,M,4000
Maria,Santanna,Italiana,Dentista,F,6000
Jeane,Andrade,Inglesa,Medica,F,7000


In [0]:
#Lendo o aquivo participonado do parquet
datafnacional=spark.read.parquet("/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Brasileira")
datafnacional.show(truncate=False)


+-------------+-----------+---------+------+-------+
|Primeiro_Nome|Ultimo_nome|Trabalho |Genero|salario|
+-------------+-----------+---------+------+-------+
|Grimaldo     |Oliveira   |Professor|M     |3000   |
|Roberto      |Carlos     |Analista |M     |4000   |
+-------------+-----------+---------+------+-------+



In [0]:
#Consultando diretamente o arquivo parquet particionado via

spark.sql("CREATE OR REPLACE TEMPORARY VIEW Cidadao USING parquet OPTIONS (path\"/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Brasileira\")")
spark.sql("SELECT * FROM Cidadao where Ultimo_nome='Oliveira'").show()

+-------------+-----------+---------+------+-------+
|Primeiro_Nome|Ultimo_nome| Trabalho|Genero|salario|
+-------------+-----------+---------+------+-------+
|     Grimaldo|   Oliveira|Professor|     M|   3000|
+-------------+-----------+---------+------+-------+

